In [ ]:
cluster_S8.draw_graph()

In [ ]:
G = cluster_S8.create_graph()
communities = list(nx.community.label_propagation_communities(G))
subgraphs = [G.subgraph(community) for community in communities]

for i, subgraph in enumerate(subgraphs):
    print(f"Subgraph {i + 1} nodes:", subgraph.nodes())


In [ ]:
layout = nx.spring_layout(subgraphs[309])
nx.draw(subgraphs[309], layout, with_labels = True, node_size = 1, node_color = 'black', edge_color = 'grey')
plt.show()

In [ ]:
nodelist = list(subgraphs[309].nodes())
subdataframe = dataframe[dataframe['sequence_id'].isin(nodelist)]
subdataframe

In [ ]:
import pandas as pd
from Bio.Seq import Seq

df = subdataframe

def translate_dna_to_amino_acid(dna_sequence):
    return str(Seq(dna_sequence).translate())

def check_alignment_and_shift(dna_sequence, amino_acid_sequence):
    shifted_dna = dna_sequence
    while True:
        translated_aa = translate_dna_to_amino_acid(shifted_dna)
        if translated_aa == amino_acid_sequence:
            return True, shifted_dna
        if len(shifted_dna) < 3:
            break
        shifted_dna = shifted_dna[1:]
    return False, None

alignment_status = []
shifted_sequences = []
for i, row in df.iterrows():
    aligned, shifted_sequence = check_alignment_and_shift(row['sequence'], row['sequence_aa'])
    alignment_status.append(aligned)
    shifted_sequences.append(shifted_sequence)

df['reading_frame_aligned'] = alignment_status
df['shifted_sequence'] = shifted_sequences

subdataframe


In [ ]:
def calc_div_from_germline(sequence):
    div_from_germline = defaultdict(list)
    for i in range(len(sequence)):
        hamming_distance = 0
        for j in range(len(sequence['sequence_alignment'].iloc[i])):
            if sequence['sequence_alignment'].iloc[i][j] != sequence['germline_alignment'].iloc[i][j]:
                hamming_distance += 1
        div_from_germline[i] = hamming_distance
    return div_from_germline

temp = calc_div_from_germline(subdataframe\
temp

In [ ]:
codontab = {
    'TCA': 'S',    # Serina
    'TCC': 'S',    # Serina
    'TCG': 'S',    # Serina
    'TCT': 'S',    # Serina
    'TTC': 'F',    # Fenilalanina
    'TTT': 'F',    # Fenilalanina
    'TTA': 'L',    # Leucina
    'TTG': 'L',    # Leucina
    'TAC': 'Y',    # Tirosina
    'TAT': 'Y',    # Tirosina
    'TAA': '*',    # Stop
    'TAG': '*',    # Stop
    'TGC': 'C',    # Cisteina
    'TGT': 'C',    # Cisteina
    'TGA': '*',    # Stop
    'TGG': 'W',    # Triptofano
    'CTA': 'L',    # Leucina
    'CTC': 'L',    # Leucina
    'CTG': 'L',    # Leucina
    'CTT': 'L',    # Leucina
    'CCA': 'P',    # Prolina
    'CCC': 'P',    # Prolina
    'CCG': 'P',    # Prolina
    'CCT': 'P',    # Prolina
    'CAC': 'H',    # Histidina
    'CAT': 'H',    # Histidina
    'CAA': 'Q',    # Glutamina
    'CAG': 'Q',    # Glutamina
    'CGA': 'R',    # Arginina
    'CGC': 'R',    # Arginina
    'CGG': 'R',    # Arginina
    'CGT': 'R',    # Arginina
    'ATA': 'I',    # Isoleucina
    'ATC': 'I',    # Isoleucina
    'ATT': 'I',    # Isoleucina
    'ATG': 'M',    # Methionina
    'ACA': 'T',    # Treonina
    'ACC': 'T',    # Treonina
    'ACG': 'T',    # Treonina
    'ACT': 'T',    # Treonina
    'AAC': 'N',    # Asparagina
    'AAT': 'N',    # Asparagina
    'AAA': 'K',    # Lisina
    'AAG': 'K',    # Lisina
    'AGC': 'S',    # Serina
    'AGT': 'S',    # Serina
    'AGA': 'R',    # Arginina
    'AGG': 'R',    # Arginina
    'GTA': 'V',    # Valina
    'GTC': 'V',    # Valina
    'GTG': 'V',    # Valina
    'GTT': 'V',    # Valina
    'GCA': 'A',    # Alanina
    'GCC': 'A',    # Alanina
    'GCG': 'A',    # Alanina
    'GCT': 'A',    # Alanina
    'GAC': 'D',    # Acido Aspartico
    'GAT': 'D',    # Acido Aspartico
    'GAA': 'E',    # Acido Glutamico
    'GAG': 'E',    # Acido Glutamico
    'GGA': 'G',    # Glicina
    'GGC': 'G',    # Glicina
    'GGG': 'G',    # Glicina
    'GGT': 'G'     # Glicina
}
aminotab = defaultdict(list)
for codon, aa in codontab.items():
    aminotab[aa].append(codon)

In [ ]:
subdataframe['sequence_len'] = subdataframe['shifted_sequence'].apply(lambda x: len(x)).copy()
sequences = list(subdataframe.groupby('sequence_len'))[1][1]
sequences[sequences['sequence'] == sequences['germline_alignment']]


In [ ]:
hamming_distance_map = defaultdict(list)

for i in range(len(sequences['sequence_id'])):
    for j in range(i+1,  len(sequences['sequence_id'])):
        hamming_distance = 0
        for k in range(len(sequences['shifted_sequence'].iloc[i])):
            if sequences['shifted_sequence'].iloc[i][k] != sequences['shifted_sequence'].iloc[j][k]:
                hamming_distance += 1
        if hamming_distance == 1:
            hamming_distance_map[sequences['sequence_id'].iloc[i]].append(sequences['sequence_id'].iloc[j])
            # hamming_distance_map[sequences['sequence_id'].iloc[j]].append(sequences['sequence_id'].iloc[i])


In [ ]:
subG = nx.Graph()

for seq, neighbors in hamming_distance_map.items():
    subG.add_node(seq)
    for neighbor in neighbors:
        subG.add_node(neighbor)
        subG.add_edge(seq, neighbor)

subG.nodes()

In [ ]:
layout = nx.spring_layout(subG)
nx.draw(subG, layout, with_labels=True, node_size=1, node_color='red', edge_color='grey', font_size = 7)
plt.show()
phylogenetic tree
igphyml

In [ ]:
pairwise_distances = nx.floyd_warshall_numpy(cluster_S8.create_graph())
plt.figure(figsize=(8,8))
plt.imshow(pairwise_distances, cmap='viridis', interpolation='none')
plt.colorbar(label='Distance')
plt.title('Pairwise Distance Matrix')
plt.show()

In [ ]:
# calculating clonal expansion index, in edges
edge_counts, gini_coefficient = cluster_S8.clonal_expansion_index_by_edges()
fig = plt.figure(figsize=(18,6))
axis1 = plt.subplot(1, 3, 1)
axis1.plot(edge_counts, 'r-', linewidth=1)
axis1.set_xlabel('Node')
axis1.set_ylabel('Degree')

# calculating clonal expansion index, in sequences
sequence_counts, gini_seq_coefficient = cluster_S8.clonal_expansion_index_by_duplicates()

axis2 = plt.subplot(1, 3, 2)
axis2.plot(sequence_counts, 'r-', linewidth=1)
axis2.set_xlabel('Sequence')
axis2.set_ylabel('Count')

# calculating clonal diversification index, in # of unique VDJ seqs in clones
cluster_size, shannon_entropy = cluster_S8.clonal_diversification_index()

axis3 = plt.subplot(1, 3, 3)
axis3.plot(cluster_size, 'r-', linewidth = 1)
axis3.set_xlabel('Clones')
axis3.set_ylabel('Size')

plt.show()

print(f'Clonal expansion index (sequence, gini) : {gini_seq_coefficient}')
print(f'Clonal expansion index (edge, gini) : {gini_coefficient}')
print(f'Clonal diversification index (clones, shannon) : {shannon_entropy}')